# Corpus Stats

In [3]:
%cd ../../

/home/mkranzlein/michael/dev/curiam


In [ ]:
import os
from collections import Counter

### Table showing total number of opinions and number of majority, concurring, and dissenting opinions

In [8]:
opinion_counts = {}
for filename in os.listdir("data/full_scale/processed/inception_files_with_docket_numbers"):
    docket_subpart_1, docket_subpart_2, opinion_type, author = filename.split("_")
    # Format authors for table
    author = author[:-4]
    author = author[0].upper() + author[1:]
    if author not in opinion_counts.keys():
        opinion_counts[author] = Counter()
    opinion_counts[author][opinion_type] += 1

authors = sorted(opinion_counts.keys())
total = 0
majority_total = 0
concurrence_total = 0
dissent_total = 0
table_output = ""
for author in authors:
    # Calculate row sums and increment column totals
    majority = opinion_counts[author]["ootc"]
    majority_total += majority
    concurrence = opinion_counts[author]["concurrence"]
    concurrence_total += concurrence
    dissent = opinion_counts[author]["dissent"]
    dissent_total += dissent
    # Total for this particular justice
    justice_total = (majority + concurrence + dissent)
    total += justice_total
    table_output += f"{author} & {majority} & {concurrence} & {dissent} & {justice_total}\\\\\n"
# Remove newline from second-to-last row so midrule can be added
table_output = table_output[:-1]
table_output += f" \\midrule\n" + f"& {majority_total} & {concurrence_total} & {dissent_total} & {total}\\\\"
with open("results/tables/opinion_characterisitcs.txt", "w") as f:
    f.write(table_output)
print(table_output)

Alito & 3 & 2 & 3 & 8\\
Breyer & 1 & 1 & 0 & 2\\
Ginsburg & 2 & 1 & 1 & 4\\
Gorsuch & 3 & 1 & 1 & 5\\
Kagan & 2 & 0 & 0 & 2\\
Kavanaugh & 2 & 2 & 1 & 5\\
Roberts & 1 & 0 & 0 & 1\\
Sotomayor & 2 & 3 & 2 & 7\\
Thomas & 2 & 1 & 4 & 7\\ \midrule
& 18 & 11 & 12 & 41\\


### How many tokens are there?

### How many spans were annotated (including overlaps)?

### What percentage of the tokens were part of a metalanguage span (no overlaps)

